In [1]:
import numpy as np
import pandas as pd
import joblib
import warnings
import os
from sklearn.preprocessing import StandardScaler


In [2]:
use_cols = joblib.load('./use_cols')
scaler = joblib.load('./scaler')
pca = joblib.load('./model/pca_1XX')

In [3]:
def load_test(train_no=1, use_cols=[]):
    input_dir = '../../2.2preprocess_test_data/concats_cut/'
    train = joblib.load(input_dir+'concat_0%d.lz4'%train_no) 
    x_train = train[use_cols]
    CLI = train['CLI']
    CL = train['CL']
    return x_train, CLI, CL

def standard_scale_new(data,scaler):
    cols = data.columns
    idx = data.index
    data_s = scaler.transform(data.values)
    return pd.DataFrame(data=data_s,columns=cols,index=idx)

def cal_T2(data,pca):
    T2=[]
    projected_data = pca.transform(data)
    eigenvalue_matrix = np.mat(np.diag(pca.singular_values_ ))
    for x in projected_data:
        tmp = np.mat(x)*np.linalg.inv(eigenvalue_matrix)*np.mat(x).T
        T2.append(tmp[0,0])
    return T2

def write_files(test_no, use_cols, scaler, pca):
    x_test, CLI, CL = load_test(test_no, use_cols)
    data_stand = standard_scale_new(x_test,scaler)
    T2 = cal_T2(data_stand,pca)
    T2 = pd.DataFrame(data=T2,columns=['PCA_T2'], index=data_stand.index)
    T2['CLI'] = CLI
    T2['CL'] = CL
    if not os.path.exists('./test_HA'):
        os.mkdir('./test_HA')
    joblib.dump(T2, './test_HA/test_HA_0%d.lz4'%test_no, compress='lz4')

In [5]:
write_files(1, use_cols, scaler, pca)
write_files(2, use_cols, scaler, pca)
write_files(3, use_cols, scaler, pca)
write_files(4, use_cols, scaler, pca)
write_files(5, use_cols, scaler, pca)